In [70]:
import numpy as np
import re

### Funkcja generująca tablicę numpy z wystarczającą liczbą wierszy dla każdego ograniczenia, funkcją celu i wystarczającą liczbą kolumn dla zmiennych, zmiennych luzu, M (maks./min.) i odpowiadającej im wartości.
variables - liczba zmiennych x1, x2... 

contraints - liczba ograniczeń x1 < x2 ...

In [71]:
def generate_matrix(variables , constraints):    
    table = np.zeros((constraints + 1, variables + constraints + 2))    
    return table

print(generate_matrix(5, 5))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


**next_round_right** - sprawdza czy wymagana jest 1+ oś obrotu z powodu ujemnego elementu w najdalszej prawej kolumnie, wyłączając dolną wartość.

**next_round_bottom** - sprawdza czy wymagana jest 1+ oś obrotu z powodu ujemnego elementu w dolnym wierszu, wyłączając wartość końcową.

table - wygenerowana w punkcie pierwszym tablica

In [72]:
def next_round_right(table):    
    m = min(table[:-1,-1])    
    if m>= 0:        
        return False    
    else:        
        return True

def next_round_bottom(table):    
    lr = len(table[:,0])   
    m = min(table[lr-1,:-1])    
    if m>=0:
        return False
    else:
        return True

### Niezależnie od tego, czy wymagane są dodatkowe elementy obrotowe, musimy określić, gdzie znajdują się te elementy.
**find_negative_right** - znajduje elementy negatywne w skrajnej prawej kolumnie.

**find_negative_bottom** - znajduje elementy negatywne w dolnym rzędzie

In [73]:
def find_negative_right(table):
    lc = len(table[0,:])
    m = min(table[:-1,lc-1])
    if m<=0:        
        n = np.where(table[:-1,lc-1] == m)[0][0]
    else:
        n = None
    return n

def find_negative_bottom(table):
    lr = len(table[:,0])
    m = min(table[lr-1,:-1])
    if m<=0:
        n = np.where(table[lr-1,:-1] == m)[0][0]
    else:
        n = None
    return n

### Znalezienie elementu obrodu odpowiadającego wartościom które znaleźliśmy w poprzednim punkcie

**local_pivot_right** - znalezienie elementu obrotu dla negatywnych elementów w ostatniej kolumnie

**local_pivot_bottom** - znalezienie elementu obrotu dla negatywnych elementów w dolnym rzędzie 

In [74]:
def local_pivot_right(table):
    total = []        
    r = find_negative_right(table)
    row = table[r,:-1]
    m = min(row)
    c = np.where(row == m)[0][0]
    col = table[:-1,c]
    for i, b in zip(col,table[:-1,-1]):
        if i**2>0 and b/i>0:
            total.append(b/i)
        else:                
            total.append(10000)
    index = total.index(min(total))        
    return [index,c]

def local_pivot_bottom(table):
    if next_round_bottom(table):
        total = []
        n = find_negative_bottom(table)
        for i,b in zip(table[:-1,n],table[:-1,-1]):
            if b/i >0 and i**2>0:
                total.append(b/i)
            else:
                total.append(10000)
        index = total.index(min(total))
        return [index,n]

### Funkcja przestawna, obraca się wokół elementu, aby usunąć negatywny wpis w ostatniej kolumnie lub wierszu i zwraca zaktualizowaną tabelę.

In [75]:
def pivot(row, col, table):
    lr = len(table[:,0])
    lc = len(table[0,:])
    t = np.zeros((lr,lc))
    pr = table[row,:]
    if table[row,col]**2>0:
        e = 1/table[row,col]
        r = pr*e
        for i in range(len(table[:,col])):
            k = table[i,:]
            c = table[i,col]
            if list(k) == list(pr):
                continue
            else:
                t[i,:] = list(k-r*c)
        t[row,:] = list(r)
        return t
    else:
        print('Cannot pivot on this element.')

# TODO Zrobić lepiej (Maciej)
### Funkcja pozwalająca na zamiane ciągu na zmienne float, 
(„1,3,L,5”); oznacza to, że 1(x1) + 3(x2) ≤ 5. Alternatywnie, „G” może oznaczać nierówność ≥.

In [76]:
# def convert(eq):
#     eq = eq.split(',')
#     if 'G' in eq:
#         g = eq.index('G')
#         del eq[g]
#         eq = [float(i)*-1 for i in eq]
#         return eq
#     if 'L' in eq:
#         l = eq.index('L')
#         del eq[l]
#         eq = [float(i) for i in eq]
#         return eq

def convert(eq):
    if re.match(r'(.*)(<=|>=)(.*)', eq):
        left = re.match(r'(.*)(<=|>=)(.*)', eq).group(1).replace(" ", "").replace("+", " +").replace("-", " -").lower()
        side = re.match(r'(.*)(<=|>=)(.*)', eq).group(2).replace(" ", "")
        right = re.match(r'(.*)(<=|>=)(.*)', eq).group(3).replace(" ", "")

        eq = []
        left = re.split(' ', left)
        if(left[0] == ''):
            left.pop(0)

        for x in left:
            x = re.match(r'^(-?\+?[0-9]*)x[0-9]*', x).group(1)
            if(x == '' or x == '+'):
                x = 1
            elif(x == '-'):
                x = -1
            eq.append(float(x))
            
        if(side == '<='):
            eq = [float(i) for i in eq]
        elif(side == '>='):
            eq = [float(i)*-1 for i in eq]
        
        eq.append(float(right))
        
        return eq
    else: 
        print('Wrong input constrain.')

# TODO Jakoś lepiej opisać że dla jednego trzeba tam tą magie z zamianą współczynników na minus (Norbert)
Aby rozwiązać problemy związane z minimalizacją, konieczne jest wykonanie następujących czynności. W problemach maksymalizacji M jest przeciwieństwem funkcji celu. Np. 4(x1) + 2(x2) staje się -4(x1) -2(x2). Jednak w przypadku problemów z minimalizacją funkcja celu pozostaje „taka, jaka jest”, ogólnie rzecz biorąc, ponieważ jest ona przeciwieństwem funkcji celu maksymalizacji.

In [77]:
def convert_min(table):
    table[-1,:-2] = [-1*i for i in table[-1,:-2]]
    table[-1,-1] = -1*table[-1,-1]    
    return table

### Funkcją generująca odpowiedną liczbę zmiennych x1, x2 itd.


In [78]:
def generate_variable(table):
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    v = []
    for i in range(var):
        v.append('x'+str(i+1))
    return v

# TODO można lepiej chyba żeby na starcie definiować ile ograniczeń a nie potem po piździe sprawdzać czy można (Maciej)
### Funkcja sprawdzająca czy do macierzy można dodać ograniczenia

In [79]:
def add_cons(table):
    lr = len(table[:,0])
    empty = []
    for i in range(lr):
        total = 0
        for j in table[i,:]:                       
            total += j**2
        if total == 0: 
            empty.append(total)
    if len(empty)>1:
        return True
    else:
        return False

### TODO: Wypierdalać eksepszyn jak nie mozna constrainta

 Funkcja wstawiająca odpowiednie dane z warunków do tablicy wygenerowanej wcześniej

In [80]:
def constrain(table, eq):
    if add_cons(table) == True:
        lc = len(table[0,:])
        lr = len(table[:,0])
        var = lc - lr -1      
        j = 0
        while j < lr:            
            row_check = table[j,:]
            total = 0
            for i in row_check:
                total += float(i**2)
            if total == 0:                
                row = row_check
                break
            j +=1
        eq = convert(eq)
        i = 0
        while i<len(eq)-1:
            row[i] = eq[i]
            i +=1        
        row[-1] = eq[-1]
        row[var+j] = 1    
    else:
        print('Cannot add another constraint.')

# TODO To też jakoś można lepiej chyba (Norbert) - jedna funkcja?
### Sprawdzenie czy można dodać funkcje celu do tablicy



In [81]:
def add_obj(table):
    lr = len(table[:,0])
    empty = []
    for i in range(lr):
        total = 0        
        for j in table[i,:]:
            total += j**2
        if total == 0:
            empty.append(total)    
    if len(empty)==1:
        return True
    else:
        return False

### Dodanie funkcji ceu do tablicy

In [82]:
def obj(table,eq):
    if add_obj(table)==True:
        eq = [float(i) for i in eq.split(',')]
        lr = len(table[:,0])
        row = table[lr-1,:]
        i = 0        
        while i<len(eq)-1:
            row[i] = eq[i]*-1
            i +=1
        row[-2] = 1
        row[-1] = eq[-1]
    else:
        print('You must finish adding constraints before the objective function can be added.')

# TODO jakoś opis zrobić dobrze, ale generalnie jest to spięcie wszystkiego co wcześniej - flagi, jedna funkcja z maxz i minz (Maciej)
użyją pętli while do określenia, czy wymagany jest element obrotu 1+, zlokalizowania elementu obrotu, jego obrotu wokół niego i kontynuowania procesu, aż wszystkie elementy ujemne zostaną usunięte z ostatniej kolumny i wiersza. Następnie zostaną wygenerowane zmienne dla x1 do xn i przypisane wartości zgodnie z ich pozycjami w tableau. Dodatkowo max otrzyma odpowiednią wartość. Na koniec funkcja zwróci max i zmienne w formie słownika.

In [83]:
def maxz(table):
    while next_round_right(table)==True:
        table = pivot(local_pivot_right(table)[0],local_pivot_right(table)[1],table)
    while next_round_bottom(table)==True:
        table = pivot(local_pivot_bottom(table)[0],local_pivot_bottom(table)[1],table)        
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    i = 0
    val = {}
    for i in range(var):
        col = table[:,i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]            
            val[generate_variable(table)[i]] = table[loc,-1]
        else:
            val[generate_variable(table)[i]] = 0
    val['max'] = table[-1,-1]
    return val

# TODO to co wyżej tylko dla minimalizacjii (robią prawie to samo)

In [84]:
def minz(table):
    # !!!Jedyna różnica
    table = convert_min(table)
    while next_round_right(table)==True:
        table = pivot(local_pivot_right(table)[0],local_pivot_right(table)[1],table)    
    while next_round_bottom(table)==True:
        table = pivot(local_pivot_bottom(table)[0],local_pivot_bottom(table)[1],table)       
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    i = 0
    val = {}
    for i in range(var):
        col = table[:,i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]             
            val[generate_variable(table)[i]] = table[loc,-1]
        else:
            val[generate_variable(table)[i]] = 0 
            val['min'] = table[-1,-1]*-1
    return val

In [90]:
    # Testy czy działa
    m = generate_matrix(2,2)
    # 2x1 -x2 >= 10
    constrain(m,'x2 - x1 <= 20')
    # x1 + x2 <= 20
    constrain(m,'x1 + 3x2 <= +10')
    obj(m,'5,10,0')
    print(maxz(m))     

    print(m)

{'x1': 10.0, 'x2': 0, 'max': 50.0}
[[  1.  -1.   1.   0.   0.  20.]
 [  1.   3.   0.   1.   0.  10.]
 [ -5. -10.   0.   0.   1.   0.]]
